<a href="https://colab.research.google.com/github/pepedp/Otros/blob/master/API_CMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title FUNCIONES 
import requests
import pandas as pd
import numpy as np
import json
!mkdir archivos 
!mkdir salida 

path= '/content/archivos/'
usuario, token = 'CMI','CMITOKEN'

#Funcion coexion API   
def API(fecha_inicio, ficha_fin, nombre_tabla, archivo=0, path=path):
  url = "https://visvivadata.com//api/db"
  
  #Conexion a la BD de 
  parametros = {'start': fecha_inicio, 'end': ficha_fin, 
        'table': nombre_tabla, 
        'user': usuario, 'token': token}
  data_json = requests.post(url, parametros).json()  
  data = pd.json_normalize(data_json["data"],"data")
  
  #Guardar archivos 
  # 0 = Guarda archivo .csv y .json 
  # 1 = Guarda archivo .csv
  # 2 = Guarda archivo .json 

  if archivo == 0:
    data.to_csv(path+nombre_tabla+'.csv',index=None)
    with open(path+nombre_tabla+'.json', 'w') as file:
      json.dump(data_json, file, indent=4)
  
  else:
    if archivo == 1:
      data.to_csv(path+nombre_tabla+'.csv',index=None)

    if archivo == 2:
      with open(path+nombre_tabla+'.json', 'w') as file:
        json.dump(data_json, file, indent=4)   

  return data


#Funcion para comprimir los archivos 
def comprimir_archivos(nombre_archivo="ArchivosVisViva",extension=".csv",directorio=path):
  import os
  import zipfile
  
  archivos_zip = zipfile.ZipFile(directorio + nombre_archivo + '.zip', 'w')
  for folder, subfolders, files in os.walk(directorio):
      for file in files:
          if file.endswith(extension):
              archivos_zip.write(os.path.join(folder, file), 
                                os.path.relpath(os.path.join(folder,file), directorio), 
                                compress_type = zipfile.ZIP_DEFLATED)  
  archivos_zip.close()

mkdir: cannot create directory ‘archivos’: File exists
mkdir: cannot create directory ‘salida’: File exists


In [ ]:
#CONSULTAR UNA TABLA
tabla = 'TransaccionesContratosEOR'
datos=API_VisViVa('25/01/2021','02/02/2021',tabla)
datos

In [ ]:
#CONSULTAR MULTIPLES TABLAS 
tablas_api=pd.read_excel("VisViva.xlsx")
tablas_api=np.array(tablas_api["tabla"])
fecha_ini, fecha_fin = '25/01/2021','02/02/2021'

for nombre_tabla in range(0,len(tablas_api) ):
  datos=API(fecha_ini, fecha_fin,tablas_api[nombre_tabla],1)
  print("Archivo Creado - Tabla: {} ({:,.0f} registros)".format(tablas_api[nombre_tabla],len(datos)) )
comprimir_archivos()
print("{} tablas procesadas ...".format(nombre_tabla+1) )